In [18]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels

import os
import random

In [19]:
import sklearn
import tensorflow 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [20]:
os.environ['PYTHONHASHSEED'] = '42'  

random.seed(42)          
np.random.seed(42)       
tensorflow.random.set_seed(42)   

In [21]:
# load the data into dataframe
df=pd.read_csv('feature_engineered_data.csv')
print(df.head())

   current_value  lights         T1       RH_1         T2       RH_2  \
0          430.0      30  20.133333  48.000000  19.566667  44.400000   
1          250.0      30  20.260000  52.726667  19.730000  45.100000   
2          100.0      10  20.426667  55.893333  19.856667  45.833333   
3          100.0      10  20.566667  53.893333  20.033333  46.756667   
4           90.0      10  20.730000  52.660000  20.166667  47.223333   

          T3       RH_3         T4       RH_4  ...  is_weekend    nsm  lag_1  \
0  19.890000  44.900000  19.000000  46.363333  ...           0  68400  576.6   
1  19.890000  45.493333  19.000000  47.223333  ...           0  69000  430.0   
2  20.033333  47.526667  19.000000  48.696667  ...           0  69600  250.0   
3  20.100000  48.466667  19.000000  48.490000  ...           0  70200  100.0   
4  20.200000  48.530000  18.926667  48.156667  ...           0  70800  100.0   

   lag_2  lag_6  lag_12  hour_sin  hour_cos  rolling_mean_12  rolling_std_12  
0  230.

In [22]:
def evaluate(model_name,target_scaler,y_test,y_pred):
   
    y_pred_actual = target_scaler.inverse_transform(y_pred)
    y_test_actual = target_scaler.inverse_transform(y_test.reshape(-1, 1))

    mae_GRU = mean_absolute_error(y_test_actual, y_pred_actual)
    r2_GRU = r2_score(y_test_actual, y_pred_actual)
    rmse_GRU = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))

    print(f"Model Name :{model_name}")
    print(f"MAE: {mae_GRU:.2f}")
    print(f"R² Score: {r2_GRU:.4f}")
    print(f"RMSE: {rmse_GRU:.4f}")

In [23]:
import pickle

# Load the selected features for 10-minute forecasting
with open("selected_features_10min.pkl", "rb") as f:
    selected_features = pickle.load(f)

print("Selected Features:", selected_features)

Selected Features: ['current_value', 'RH_5', 'T6', 'T8', 'RH_8', 'Press_mm_hg', 'nsm', 'lag_1', 'rolling_mean_12', 'rolling_std_12']


In [24]:
df['target_10min']= df['current_value'].shift(-1)

In [25]:
df = df.dropna().reset_index(drop=True)

In [26]:
df = df[selected_features + ['target_10min']]

In [27]:
train_size = int(0.8 * len(df))
df_train, df_test = df[:train_size], df[train_size:]

In [28]:
scaler = MinMaxScaler()
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train), columns=df.columns, index=df_train.index)
df_test_scaled = pd.DataFrame(scaler.transform(df_test), columns=df.columns, index=df_test.index)


In [29]:
def create_sequences(data, target_column, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data.iloc[i-window_size:i].values)
        y.append(data.iloc[i][target_column])
    return np.array(X), np.array(y)

window_size = 24  # use previous 24 time steps
#X, y = create_sequences(scaled_df, target_column='target_10min', window_size=window_size)
X_train, y_train = create_sequences(df_train_scaled, target_column='target_10min', window_size=window_size)
X_test, y_test = create_sequences(df_test_scaled, target_column='target_10min', window_size=window_size)


In [30]:
# Invert scaling for target only
target_scaler = MinMaxScaler()
target_scaler.fit(df[['target_10min']])  # fit only on original (unscaled) appliances column

MinMaxScaler()

# LSTM

In [31]:
lstm = Sequential()
lstm.add(LSTM(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
lstm.add(Dense(1))  # Output layer

lstm.compile(optimizer='adam', loss='mse',metrics=['mae'])
history = lstm.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, shuffle=False)


Epoch 1/20


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


444/444 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0299 - mae: 0.1050 - val_loss: 0.0130 - val_mae: 0.0524
Epoch 2/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0182 - mae: 0.0768 - val_loss: 0.0108 - val_mae: 0.0468
Epoch 3/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0152 - mae: 0.0671 - val_loss: 0.0097 - val_mae: 0.0450
Epoch 4/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0144 - mae: 0.0637 - val_loss: 0.0096 - val_mae: 0.0450
Epoch 5/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0142 - mae: 0.0624 - val_loss: 0.0095 - val_mae: 0.0451
Epoch 6/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0141 - mae: 0.0618 - val_loss: 0.0094 - val_mae: 0.0452
Epoch 7/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0140 - mae: 0.0615 - val_loss: 0.0094 - val_mae: 0.0451
Epoch 8/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0139 - mae: 0.0612 - val_loss: 0.0094 - val_mae: 0.0448
Epoch 9/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0139 

In [32]:
# Predict
y_pred_lstm = lstm.predict(X_test)

evaluate("LSTM",target_scaler,y_test,y_pred_lstm)

123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Model Name :LSTM
MAE: 26.77
R² Score: 0.5709
RMSE: 55.6030


# GRU

In [33]:
from tensorflow.keras.layers import GRU


gru = Sequential()
gru.add(GRU(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
gru.add(Dense(1))  # Output layer

gru.compile(optimizer='adam', loss='mse',metrics=['mae'])
history = gru.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, shuffle=False)


Epoch 1/20


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


444/444 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0222 - mae: 0.0880 - val_loss: 0.0117 - val_mae: 0.0551
Epoch 2/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0153 - mae: 0.0678 - val_loss: 0.0100 - val_mae: 0.0461
Epoch 3/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0147 - mae: 0.0651 - val_loss: 0.0097 - val_mae: 0.0446
Epoch 4/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0145 - mae: 0.0641 - val_loss: 0.0096 - val_mae: 0.0445
Epoch 5/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0144 - mae: 0.0634 - val_loss: 0.0095 - val_mae: 0.0449
Epoch 6/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0143 - mae: 0.0628 - val_loss: 0.0094 - val_mae: 0.0457
Epoch 7/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0142 - mae: 0.0623 - val_loss: 0.0094 - val_mae: 0.0463
Epoch 8/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0141 - mae: 0.0620 - val_loss: 0.0094 - val_mae: 0.0464
Epoch 9/20
444/444 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0140 

In [ ]:
# Predict
y_pred_gru = gru.predict(X_test)


evaluate("GRU",target_scaler,y_test,y_pred_gru)

123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Model Name :GRU
MAE: 28.04
R² Score: 0.5701
RMSE: 55.6545


In [35]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tcn import TCN  # keras-tcn package

In [36]:
model_tcn = Sequential([
    TCN(input_shape=(X_train.shape[1], X_train.shape[2])),  # (timesteps, features)
    Dense(1)
])

model_tcn.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_tcn.summary()

model_tcn.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1,shuffle=False)


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\tcn\tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tcn (TCN)                       │ (None, 64)             │       138,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,881 (542.50 KB)

 Trainable params: 138,881 (542.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 1.4308 - mae: 0.5912 - val_loss: 0.0224 - val_mae: 0.1071
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0362 - mae: 0.1399 - val_loss: 0.0176 - val_mae: 0.0890
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.0264 - mae: 0.1160 - val_loss: 0.0151 - val_mae: 0.0790
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0234 - mae: 0.1063 - val_loss: 0.0156 - val_mae: 0.0817
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0216 - mae: 0.1006 - val_loss: 0.0218 - val_mae: 0.1152
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0217 - mae: 0.1022 - val_loss: 0.0346 - val_mae: 0.1641
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0205 - mae: 0.0981 - val_loss: 0.0251 - val_mae: 0.1349
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0174 - mae: 0.0869 - val_loss: 0.0235 - val_mae: 0.1292
Epoch 9/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/

In [37]:
y_pred_tcn = model_tcn.predict(X_test)

evaluate("TCN",target_scaler,y_test,y_pred_tcn)

123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Model Name :TCN
MAE: 107.57
R² Score: -0.9948
RMSE: 119.8829


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D, Flatten, LSTM, Dense


In [39]:
model_cnn = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model_cnn.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_cnn.summary()


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 22, 64)         │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 704)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │        35,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,477 (146.39 KB)

 Trainable params: 37,477 (146.39 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model_cnn.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1,shuffle=False)


Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0382 - mae: 0.1239 - val_loss: 0.0162 - val_mae: 0.0762
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0237 - mae: 0.0920 - val_loss: 0.0135 - val_mae: 0.0663
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0199 - mae: 0.0832 - val_loss: 0.0121 - val_mae: 0.0588
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0177 - mae: 0.0759 - val_loss: 0.0113 - val_mae: 0.0515
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0168 - mae: 0.0731 - val_loss: 0.0112 - val_mae: 0.0508
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0164 - mae: 0.0725 - val_loss: 0.0110 - val_mae: 0.0499
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0160 - mae: 0.0704 - val_loss: 0.0112 - val_mae: 0.0496
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0158 - mae: 0.0699 - val_loss: 0.0108 - val_mae: 0.0495
Epoch 9/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

In [42]:
y_pred_cnn = model_cnn.predict(X_test)
evaluate("CNN",target_scaler,y_test,y_pred_cnn)


123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step
Model Name :CNN
MAE: 29.09
R² Score: 0.5284
RMSE: 58.2884
